In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath_other = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw data other/'

In [3]:
#specify where to read raw datafiles and where to save processed files
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'
filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/'

#BAG register
sourceBAG = 'https://data.nlextract.nl/bag/csv/'
df = pd.read_csv(filepath_raw + 'bagadres-full_2019.csv', delimiter = ';' , usecols = ['x','y','postcode','huisnummer','gemeente'])#, low_memory = False)

In [4]:
#Needed to prevent errors in Vensim and Netlogo, which don't deal well with non-latins
df.gemeente = df.gemeente.replace({"Nuenen, Gerwen en Nederwetten": "Nuenen Gerwen en Nederwetten", "Noardeast-Fryslân": "Noardeast-Fryslan" , "Súdwest-Fryslân":"Sudwest-Fryslan" })

In [5]:
buurten = pd.read_csv(filepath_raw + 'postcode_huisnummer_buurten.csv',delimiter = ';')
wijk = pd.read_csv(filepath_raw + 'wijk2019.csv', delimiter = ';')
buurt = pd.read_csv(filepath_raw + 'buurt2019.csv', delimiter = ';')

In [6]:
wijk_dict = wijk.astype(str).set_index('Wijkcode2019').to_dict()['Wijknaam_2019K_NAAM']
buurt.Buurtcode_2019 = buurt.Buurtcode_2019.astype(str)
buurt_dict = buurt.set_index('Buurtcode_2019').to_dict()['Buurtnaam_2019']

buurten[['Buurt2019']]= buurten[['Buurt2019']].astype(str)
buurten[['Wijk2019']] = buurten[['Wijk2019']].astype(str)
buurten['buurt_naam'] = buurten['Buurt2019'].map(buurt_dict)
buurten['wijk_naam'] = buurten['Wijk2019'].map(wijk_dict)
buurten = buurten.drop(['Buurt2019','Wijk2019','Gemeente2019'], axis = 1).rename(columns = {'PC6':'postcode','Huisnummer':'huisnummer'})

In [7]:
df = df.merge(buurten, on = ['postcode','huisnummer'])


In [8]:
xycor = df.groupby(by = ['gemeente','buurt_naam']).mean().drop(['huisnummer'],axis = 1)

In [9]:
xycor.to_csv(filepath_processed + 'xycor_buurten.csv')

In [10]:
relative_xycor = ((xycor - xycor.min()) / (xycor.max() - xycor.min()))
          

In [11]:
relative_xycor.to_csv(filepath_processed + 'relative_xycor_buurten.csv')

In [12]:
MT_sources = pd.read_csv(filepath_other + '20190802_puntbronnen_MT.csv', delimiter = ';')

In [13]:
MT_sources['xcor'] = (MT_sources.X - xycor.x.min()) / (xycor.x.max() - xycor.x.min())
MT_sources['ycor'] = (MT_sources.Y - xycor.y.min()) / (xycor.y.max() - xycor.y.min())

In [14]:
MT_sources_relative = MT_sources.drop(["X","Y"], axis = 1)

In [15]:
MT_sources_relative.MWcapaciteit.fillna(0, inplace = True)
MT_sources_relative.Brontemperatuur.fillna(70, inplace = True)
MT_sources_relative.Uitkoppel_min.fillna(MT_sources_relative.Uitkoppel_min.mean(), inplace = True)
MT_sources_relative.Uitkoppel_max.fillna(250, inplace = True)
MT_sources_relative.Productiekosten.fillna(MT_sources_relative.Productiekosten.mean(), inplace = True)

In [16]:
MT_sources_relative.to_csv(filepath_processed + 'MT_sources_relativexycor.csv')

In [17]:
MT_sources_relative

,bron_naam,type_bron,MWcapaciteit,capfactor,volfactor,status,Brontemperatuur,Uitkoppel_min,Uitkoppel_max,Productiekosten,targetGMC,xcor,ycor
0,Onbekend Heerlen,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.692455,0.046779
1,Onbekend Eindhoven 1,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.551742,0.259702
2,Onbekend Eindhoven 2,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.536946,0.248284
3,Onbekend Den Bosch,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.509477,0.340561
4,Onbekend Hulst,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.183297,0.196384
5,Onbekend Rotterdam,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.281334,0.388344
6,Onbekend Zandvoort,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.314992,0.594265
7,Onbekend Zaanstad,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.372381,0.631048
8,Onbekend Zeewolde,Gasmotor,0.50,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.573179,0.581824
9,A van de Groep en Zonen BV,NaN,0.00,NaN,NaN,Optie B,70.0,220.254545,250.0,2.009091,NaN,0.536528,0.547386


In [18]:
(xycor.x.max() - xycor.x.min()) /  (xycor.y.max() - xycor.y.min())
# RELATIVE X Y SIZE NETHERLANDS

0.8671645487815568

In [19]:
xycor.y.max() - xycor.y.min()


303183.9570003321